In [1]:
# from platform import python_version
# print(python_version())

In [2]:
# !pip install tensorflow==1.13.1
# !pip install Sequential
# !pip install keras

In [3]:
import tensorflow as tf
from tensorflow import data
from tensorflow.data import Dataset

from tensorflow import keras 
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

import os
import json
import math
import numpy as np

import sentiment_dataset as sentiment_dataset
import sentiment_model_cnn as sentiment_model_cnn
import config_holder as config_holder

Using TensorFlow backend.


In [4]:
config_file = json.load(open('training_config.json', "r"))
num_epoch= 10
train_location = 's3://yujiawang/ieor4577-hw4/train data preprocessed'
dev_location   = 's3://yujiawang/ieor4577-hw4/dev data preprocessed'
eval_location = 's3://yujiawang/ieor4577-hw4/eval data preprocessed'
model_output_dir = 's3://ieor4577-hw4/'

In [5]:
print("Preparing for training...")

training_config = config_file

training_config["num_epoch"] = num_epoch

train_dataset = sentiment_dataset.train_input_fn(train_location, training_config)
validation_dataset = sentiment_dataset.validation_input_fn(dev_location, training_config)
eval_dataset = sentiment_dataset.eval_input_fn(eval_location, training_config)

model = sentiment_model_cnn.keras_model_fn(None, training_config)

print("Starting training...")
validation_dataset[1]
model.fit(
    x=train_dataset[0], y=train_dataset[1], steps_per_epoch=train_dataset[2]["num_batches"],
    epochs=training_config["num_epoch"],
    validation_data=(validation_dataset[0], validation_dataset[1]),
    validation_steps=int(validation_dataset[2]["num_batches"]))

score = model.evaluate(
    eval_dataset[0], eval_dataset[1], steps=eval_dataset[2]["num_batches"], verbose=0)

print("Test loss:{}".format(score[0]))
print("Test accuracy:{}".format(score[1]))

Preparing for training...
Fetching train data...
Fetching validation data...
Fetching eval data...
Instructions for updating:
Colocations handled automatically by placer.
Starting training...
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
50/50 [==============================] - 1s 15ms/step - loss: 0.6916 - acc: 0.5246 - val_loss: 0.6886 - val_acc: 0.5246
Epoch 2/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6711 - acc: 0.5535 - val_loss: 0.6290 - val_acc: 0.6731
Epoch 3/10
50/50 [==============================] - 0s 6ms/step - loss: 0.5764 - acc: 0.7449 - val_loss: 0.5203 - val_acc: 0.8193
Epoch 4/10
50/50 [==============================] - 0s 6ms/step - loss: 0.4736 - acc: 0.8262 - val_loss: 0.4122 - val_acc: 0.8417
Epoch 5/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3792 - acc: 0.8441 - val_loss: 0.3379 - val_acc: 0.8539
Epoch 6/10
50/50 [=============

In [6]:
sentiment_model_cnn.save_model(model)

Model successfully saved
